# Visualising Spatial Data with Bokeh - Quickstart

In this tutorial we will explore alternatives for visualising geospatial data, using Bokeh. With Bokeh you can create interactive dashboards for displaying different types of data, spatial or not.

In [41]:
import numpy as np
import geopandas as gpd   

from pyproj import CRS  
import xyzservices.providers as xyz  
 
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, save, output_notebook, show
from bokeh.models.mappers import LinearColorMapper
from bokeh.palettes import Viridis256  

For plotting geospatial data, Bokeh requires a specific format, which involves transforming the geometries of the original GeoDataFrame into two columns recording the x and y coordinates of each point in the geometry of the GeoDataFrame.

In [42]:
def getPolyCoords(row, geom, coord_type):
    
    if row[geom].geom_type == "MultiPolygon":
        g_obj = row[geom].geoms[0]
    else:
        g_obj = row[geom] 
    
    if coord_type == 'x':
        return list(g_obj.exterior.coords.xy[0])
    elif coord_type == 'y':
        return list(g_obj.exterior.coords.xy[1])

def transform_gdf(gdf):
    
    gdf['x'] = gdf.apply(getPolyCoords, geom = 'geometry', coord_type = 'x', axis = 1)
    gdf['y'] = gdf.apply(getPolyCoords, geom = 'geometry', coord_type = 'y', axis = 1)
    p_df = gdf.drop('geometry', axis = 1).copy()
    
    return p_df

For displaying Bokeh apps in Jupyer notebooks, you must first call the command below.

In [ ]:
output_notebook()

Our first example of displaying geospatial data is showing the selected neighbohoords of Bremen, using as basemap OpenStreetMap tiles. This basemap uses Mercator coordinates, and we will first transform the input spatial data to the new coordinate system.

In [45]:
neighborhoods_bremen = gpd.read_file("../data_example/Example_Bremen_Neighborhoods.gpkg")

proj_crs = CRS.from_user_input(3035) 
mercator_crs = CRS.from_user_input(3395)

neighborhoods_bremen_copy = neighborhoods_bremen.to_crs(mercator_crs)     

We extract the boundary of the input spatial data, in order to create a map that fits its size.

In [46]:
bbox = neighborhoods_bremen_copy.total_bounds
x_range = (bbox[0], bbox[2]) ; y_range = (bbox[1], bbox[3]) 

After processing of the original GeoDataFrame is finished, we can transform the geometry column to fit Bokeh input format.

In [ ]:
neighborhoods_bremen_b = transform_gdf(neighborhoods_bremen_copy) 
neighborhoods_bremen_b.head(3)

The simple display below uses as input boundaries of the map, the spatial data to be shown, and display parameters, i.e. line color, line width and transparency.

In [ ]:
p = figure(title = "Neighborhoods in Bremen", x_range=x_range, y_range=y_range,
           x_axis_type="mercator", y_axis_type="mercator")
     
p.patches('x', 'y', source = ColumnDataSource(neighborhoods_bremen_b), line_color = "red", line_width = 0.8, 
              fill_alpha = 0)
 
p.add_tile(xyz.OpenStreetMap.Mapnik)
show(p)   


Alternatively, the map can be saved locally.

In [ ]:
f = "Plot_neighborhoods_Bremen.html"   
outfp =  f     
save(p, outfp)

In our second example, we will display spatial data using a color map.

In [49]:
zensus_bremen_grid = gpd.read_file("../data_example/Example_Bremen_Zensus_Grid_100m.gpkg")

proj_crs = CRS.from_user_input(3035) 
mercator_crs = CRS.from_user_input(3395)

zensus_bremen_grid_copy = zensus_bremen_grid.to_crs(mercator_crs) 

bbox = zensus_bremen_grid_copy.total_bounds
x_range = (bbox[0], bbox[2]) ; y_range = (bbox[1], bbox[3]) 

We create a new collumn in the GeoDataFrame, which will be used for display. The colors are chosen as random integers. Ideally, these values should not be random and instead be read from sociodemographic variables. Integers are then transformed into corresponding colors from a native color palette.

In [50]:
color_nb = 8
zensus_bremen_grid_copy["fill_col"] = np.random.randint(0, color_nb, len(zensus_bremen_grid_copy)) 
color_mapper =  LinearColorMapper(palette=Viridis256, low=0, high = color_nb)  

Transforming the geometries, displaying the map and saving it, are the same as in the previous example.

In [51]:
zensus_bremen_grid_b = transform_gdf(zensus_bremen_grid_copy)

In [ ]:
p = figure(title = "Zensus Grid in Bremen", x_range=x_range, y_range=y_range,
           x_axis_type="mercator", y_axis_type="mercator")
 
p.patches('x', 'y', source = ColumnDataSource(zensus_bremen_grid_b), line_color = "white", line_width = 0.5, 
              fill_color={'field': 'fill_col', 'transform': color_mapper})

p.add_tile(xyz.OpenStreetMap.Mapnik)

show(p)

In [ ]:
f = "Plot_Zensus_Bremen_colors.html"   
outfp =  f     
save(p, outfp)